In [4]:
import itk
import os
from itkwidgets import view

In [20]:
img = itk.imread('./spleen_10.nii.gz')
print(img)
print(dict(img))

Image (00000226084BC440)
  RTTI typeinfo:   class itk::Image<float,3>
  Reference Count: 1
  Modified Time: 2944
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (none)
  Source output name: (none)
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 2758
  UpdateMTime: 2943
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 55]
  BufferedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 55]
  RequestedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 55]
  Spacing: [0.976562, 0.976562, 5]
  Origin: [499.023, 499.023, 0]
  Direction: 
-1 0 0
0 -1 0
0 0 1

  IndexToPointMatrix: 
-0.976562 0 0
0 -0.976562 0
0 0 5

  PointToIndexMatrix: 
-1.024 0 0
0 -1.024 0
0 0 0.2

  Inverse Direction: 
-1 0 0
0 -1 0
0 0 1

  PixelContainer: 
    ImportImageContainer (0000022605789AF0)
      RTTI typeinfo:   class itk::ImportImageContainer<unsigned __int64,flo

In [6]:
minmax = itk.MinimumMaximumImageCalculator[itk.Image[itk.F, 3]].New(Image=img)
minmax.Compute()
print(minmax.GetMinimum(), minmax.GetMaximum())

os.makedirs('tmp', exist_ok=True)
itk.imwrite(img, './tmp/tmp.dcm')

-1024.0 3072.0


RuntimeError: C:\P\IPP\ITK-source\ITK\Modules\IO\GDCM\src\itkGDCMImageIO.cxx:1204:
itk::ERROR: itk::ERROR: GDCMImageIO(000002267C545310): A Floating point buffer was passed but the stored pixel type was not specified.This is currently not supported

In [14]:
import numpy as np
def float_image_to_dicom(img, fp):
    '''
    Parameters
    ==========
    img (itk.Image[itk.F, 3]) : input image
    fp (str) : output file path
    '''
    gdcm2 = itk.GDCMImageIO.New()

    rescaler = itk.RescaleIntensityImageFilter.New(Input=img, OutputMinimum=0, OutputMaximum=itk.NumericTraits.US.max())
    rescaler.Update()
    rescaled_cast = rescaler.GetOutput().astype(itk.US)
    img_intercept = -rescaler.GetShift()
    img_scale = rescaler.GetScale()
    print('intercept(shift): {}, scale: {}'.format(img_intercept, img_scale))

    npimg = itk.array_from_image(img)
    print('img min: {}, img max: {}'.format(np.min(npimg), np.max(npimg)))
    intercept_tag = '0028|1052'
    scale_tag = '0028|1053'
    
    rescaled_cast[intercept_tag] = str(img_intercept)
    rescaled_cast[scale_tag] = str(img_scale)
    
    # consider also bits allocated, bits stored, pixel representation, sample per pixel
#     meta = gdcm2.GetMetaDataDictionary()
#     meta.Set(intercept_tag, itk.MetaDataObject[str].New(MetaDataObjectValue=str(img_intercept)))
#     meta.Set(scale_tag, itk.MetaDataObject[str].New(MetaDataObjectValue=str(img_scale)))

    itk.imwrite(rescaled_cast, fp)
#     writer = itk.ImageSeriesWriter[itk.Image[itk.US,3],itk.Image[itk.US,3]].New(ImageIO=gdcm2, FileName=fp, Input=caster)
#     writer.Update()
    
float_image_to_dicom(img, './tmp/tmp.dcm')

intercept(shift): -16383.75, scale: 15.999755859375
img min: -1024.0, img max: 3072.0


In [15]:
img2 = itk.imread('./tmp/tmp.dcm')
#img2 = itk.imread('./tmp/tmp.dcm', pixel_type=itk.US)
print(img2)

minmax = itk.MinimumMaximumImageCalculator[itk.Image[itk.D, 3]].New(Image=img2)
minmax.Compute()
print('img2 min: {}, max: {}'.format(minmax.GetMinimum(), minmax.GetMaximum()))

Image (00000226084B9FB0)
  RTTI typeinfo:   class itk::Image<double,3>
  Reference Count: 1
  Modified Time: 2567
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (none)
  Source output name: (none)
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 2387
  UpdateMTime: 2566
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 55]
  BufferedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 55]
  RequestedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 55]
  Spacing: [0.976562, 0.976562, 5]
  Origin: [499.023, 499.023, 0]
  Direction: 
-1 0 0
0 -1 0
0 0 1

  IndexToPointMatrix: 
-0.976562 0 0
0 -0.976562 0
0 0 5

  PointToIndexMatrix: 
-1.024 0 0
0 -1.024 0
0 0 0.2

  Inverse Direction: 
-1 0 0
0 -1 0
0 0 1

  PixelContainer: 
    ImportImageContainer (00000226057872F0)
      RTTI typeinfo:   class itk::ImportImageContainer<unsigned __int64,do

In [16]:
dict(img2)

{'0008|0016': '1.2.840.10008.5.1.4.1.1.7.3',
 '0008|0018': '1.2.826.0.1.3680043.2.1125.1.44512228896057025004892280792336607',
 '0008|0020': '20210218',
 '0008|0030': '103004.751357 ',
 '0008|0050': '',
 '0008|0060': 'OT',
 '0008|0090': '',
 '0010|0010': '',
 '0010|0020': '',
 '0010|0030': '',
 '0010|0040': '',
 '0020|000d': '1.2.826.0.1.3680043.2.1125.1.27003348368731440868281022973358385',
 '0020|000e': '1.2.826.0.1.3680043.2.1125.1.78911700393739106557370434891287440',
 '0020|0010': '',
 '0020|0011': '',
 '0020|0013': '',
 '0020|0052': '1.2.826.0.1.3680043.2.1125.1.28123186485110197577539542173366064',
 '0028|0002': '1',
 '0028|0004': 'MONOCHROME2 ',
 '0028|0008': '55',
 '0028|0009': '(5200,9230)',
 '0028|0010': '512',
 '0028|0011': '512',
 '0028|0100': '16',
 '0028|0101': '16',
 '0028|0102': '15',
 '0028|0103': '0',
 '0028|1052': '-16383.75 ',
 '0028|1053': '15.999755859375 ',
 '0028|1054': 'US',
 'origin': array([  0.        , 499.02319336, 499.02319336]),
 'spacing': array([5.   

In [19]:
npimg2 = itk.array_from_image(img2)
print('dtype: {}, min: {}, max: {}'.format(npimg2.dtype, np.min(npimg2), np.max(npimg2)))

dtype: float64, min: 0.0, max: 65535.0
